In [1]:
import os
import subprocess
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from Bio import SeqIO
import pprint

### Делаем словарь со списком геномов и словарь белков

In [2]:
genomes_dict = dict()
for genus in os.listdir('data'):
    files = os.listdir('data/'+genus)
    files = list(filter(lambda x: x.endswith('.faa'), files))
    species = list(map(lambda x: x.split(".")[0], files))
    genomes_dict[genus] = species

In [26]:
aa_seqs = dict()
for genus in genomes_dict.keys():
    for genome in genomes_dict[genus]:
        aa_seqs.update(SeqIO.to_dict(SeqIO.parse(f"data/{genus}/{genome}.faa", "fasta")))

In [133]:
# Список всех геномов
genomes_list = []
for key in genomes_dict.keys():
    for genome in genomes_dict[key]:
        genomes_list.append(genome)

## Определяем кластеры

In [14]:
folders = ["data/" + folder + "/*" for folder in os.listdir("data")]
" ".join(folders)

'data/Halomonas/* data/Enterobacter/* data/Mannheimia/* data/Aeromonas/* data/Erwinia/* data/Lysobacter/* data/Klebsiella/* data/Dickeya/* data/Kosakonia/* data/Pseudomonas/*'

In [17]:
subprocess.run("proteinortho5 -project=gammaproteo " + " ".join(folders), shell=True, check=True)

CompletedProcess(args='proteinortho5 -project=gammaproteo data/Halomonas/* data/Enterobacter/* data/Mannheimia/* data/Aeromonas/* data/Erwinia/* data/Lysobacter/* data/Klebsiella/* data/Dickeya/* data/Kosakonia/* data/Pseudomonas/*', returncode=0)

In [18]:
! mv gammaproteo* proteinortho/

## Работаем со статистикой

In [ ]:
clusters = pd.read_csv("proteinortho/gammaproteo.proteinortho", sep='\t')
clusters = clusters.loc[clusters['# Species'] >= 25]
clusters.replace(to_replace='*', value=np.NaN, inplace=True)
clusters

#### Рисуем гистограмму распределения родов, видов и генов в кластерах

In [126]:
! mkdir -p pictures
sns.set_theme()
sns.histplot(data=clusters, x="# Species", discrete=True)
plt.title("Distribution of species in clusters")
plt.tight_layout()
plt.savefig(f"pictures/species_distribution.png", dpi=400)
plt.close()

sns.histplot(data=clusters, x="Genes", discrete=True)
plt.title("Distribution of genes in clusters")
plt.tight_layout()
plt.savefig(f"pictures/genes_distribution.png", dpi=400)
plt.close()

sns.histplot(data=cluster_stat, x="Genuses", discrete=True)
plt.title("Distribution of genuses in clusters")
plt.tight_layout()
plt.savefig(f"pictures/genuses_distribution.png", dpi=400)
plt.close()

In [4]:
'''
aa_seqs = dict()
for genus in genomes_dict.keys():
    temp_dict = dict()
    for genome in genomes_dict[genus]:
        temp_dict[genome] = SeqIO.to_dict(SeqIO.parse(f"data/{genus}/{genome}.faa", "fasta"))
    aa_seqs[genus] = temp_dict
del temp_dict
'''

#### Готовим таблицу

In [50]:
def return_function_by_id(ID):
    Seq = aa_seqs[ID]
    descr = Seq.description
    if ":" in descr:
        return descr[descr.find(":")+1 : descr.find("[")].strip().capitalize()
    else:
        return descr[descr.find(" ")+1 : descr.find("[")].strip().capitalize()

return_function_by_id('BBW73724.1')


'Hypothetical protein thokle011_00140'

In [89]:
def return_function_of_cluster(row):
    row = row[3:53]
    row = row.dropna()
    # To deal with multiple genes in one genome
    row = ",".join(row).split(',')
    list_of_functions = list(map(return_function_by_id, row))
    values, counts = np.unique(list_of_functions, return_counts=True)
    return values[np.argmax(counts)]

return_function_of_cluster(clusters.iloc[8])

'50s ribosomal protein l29'

In [117]:
def number_of_genuses(row):
    row = row[3:53]
    row = row.dropna()
    names = list(map(lambda x: x.split('.')[0], row.index))
    counter = 0
    for key in genomes_dict:
        for name in names:
            if name in genomes_dict[key]:
                counter += 1
                break
    return counter

number_of_genuses(clusters.iloc[10])

8

In [134]:
def read_zdna_df(genome):
    zdna = pd.read_csv(f"intersect_bed/{genome}.intersect.bed", sep='\t', 
                names=["Sp1", "St1", "En1", "Protein", "Sp2", "Start", "End", f"Score_{genome}"]
                )[["Protein", f"Score_{genome}", "Start", "End"]]
    zdna.sort_values(f"Score_{genome}", ascending=False, inplace=True)
    # Drop duplicate proteins, keep maximal score
    zdna.drop_duplicates(subset='Protein', inplace=True)
    zdna.set_index("Protein", inplace=True)
    return zdna

In [181]:
# Пересекаем Z-DNA и кластеры

for genome in genomes_list:
    zdna = read_zdna_df(genome)[f"Score_{genome}"]
    list_of_zhscores = []

    for prot in clusters[f"{genome}.faa"]:

        if pd.isnull(prot):
            list_of_zhscores.append(np.NaN)
            continue
        prot_list = prot.split(',')
        if len(prot_list) == 1:
            if prot[0] in list(zdna.index):
                list_of_zhscores.append(float(zdna.loc[prot_list[0]]))
            else:
                list_of_zhscores.append(np.NaN)
        else:
            max_score = 0
            for pr in prot_list:
                if pr in list(zdna.index):
                    max_score = max(max_score, float(zdna.loc[pr]))
            if max_score != 0:
                list_of_zhscores.append(max_score)
            else:
                list_of_zhscores.append(np.NaN)
    clusters[f"{genome}.score"] = list_of_zhscores
clusters.head()

,# Species,Genes,Alg.-Conn.,M_ovis.faa,P_citronellolis.faa,M_granulomatis.faa,M_pernigra.faa,M_bovis.faa,M_haemolytica.faa,L_caseinilyticus.faa,...,E_chengduensis.score,E_oligotrophicus.score,E_sichuanensis.score,E_ludwigii.score,E_soli.score,A_media.score,A_encheleia.score,A_hydrophila.score,A_dhakensis.score,A_caviae.score
1,27,27,1.850,WP_005539416.1,WP_003093734.1,NaN,NaN,NaN,NaN,WP_213435471.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,29,29,1.720,WP_005596065.1,WP_009617012.1,NaN,NaN,NaN,WP_006249241.1,WP_027083463.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34,34,1.470,WP_005596075.1,WP_003099086.1,WP_165889142.1,NaN,NaN,WP_006249240.1,WP_036169932.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,28,28,0.611,WP_005599073.1,WP_003457994.1,NaN,NaN,WP_025236987.1,NaN,WP_213437440.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,35,35,1.430,WP_005599322.1,WP_009617178.1,NaN,NaN,NaN,NaN,WP_213435455.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [218]:
def return_genes_with_zdna(row):
    row = row[53:103]
    row = row.dropna()
    return len(row)

def return_zdna_max(row):
    row = row[53:103]
    row = row.dropna()
    return np.max(row)

def return_zdna_mean(row):
    row = row[53:103]
    row = row.dropna()
    return np.mean(row)



(18, 23787.35, 6762.253066666668)

In [225]:
cluster_stat = pd.DataFrame(clusters[['# Species', 'Genes']])
cluster_stat['Genuses'] = clusters.apply(number_of_genuses, axis=1)
cluster_stat['Genes_with_ZDNA'] = clusters.apply(return_genes_with_zdna, axis=1)
cluster_stat['Max ZH-Score'] = clusters.apply(return_zdna_max, axis=1)
cluster_stat['Mean ZH-Score'] = clusters.apply(return_zdna_mean, axis=1)
cluster_stat['Function'] = clusters.apply(return_function_of_cluster, axis=1)
cluster_stat.sort_values(['Genes_with_ZDNA', '# Species'], ascending=False, inplace=True)
cluster_stat.to_csv("all_cluster_stats.tsv", sep='\t')
cluster_stat

,# Species,Genes,Genuses,Genes_with_ZDNA,Max ZH-Score,Mean ZH-Score,Function
977,50,73,10,18,211093.90,35448.163050,Cysteine desulfurase
1391,48,69,10,18,23787.35,6762.253067,Sec-independent protein translocase subunit tata
208,42,67,10,17,29467.88,3264.725259,P-ii family nitrogen regulator
481,42,75,10,16,9852.15,2899.237737,Hth-type transcriptional repressor purr
560,49,88,10,15,29467.88,11363.642760,Lipoprotein-releasing abc transporter permease...
...,...,...,...,...,...,...,...
6913,25,25,6,0,NaN,NaN,Transporter substrate-binding domain-containin...
7037,25,26,7,0,NaN,NaN,Type ii secretion system minor pseudopilin gsph
7039,25,25,7,0,NaN,NaN,Type ii secretion system minor pseudopilin gspj
7070,25,25,7,0,NaN,NaN,Methionine synthase


### Выбираем 5 кластеров с Z-DNA в промоторе

In [226]:
chosen = clusters.dropna(subset=clusters.columns[-50:], thresh=15)

In [227]:
chosen

,# Species,Genes,Alg.-Conn.,M_ovis.faa,P_citronellolis.faa,M_granulomatis.faa,M_pernigra.faa,M_bovis.faa,M_haemolytica.faa,L_caseinilyticus.faa,...,E_chengduensis.score,E_oligotrophicus.score,E_sichuanensis.score,E_ludwigii.score,E_soli.score,A_media.score,A_encheleia.score,A_hydrophila.score,A_dhakensis.score,A_caviae.score
208,42,67,0.114,WP_159628689.1,"WP_009622545.1,WP_003457590.1","WP_165888676.1,WP_027073914.1",WP_176807709.1,WP_025236863.1,WP_006249991.1,WP_213437086.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,42,75,0.152,"WP_159629128.1,WP_159630056.1",WP_237143049.1,"WP_165888754.1,WP_165888445.1","WP_176808502.1,WP_176807464.1","WP_188157416.1,WP_188156386.1","WP_006248949.1,WP_006249850.1",WP_213434935.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9852.15
560,49,88,0.125,"WP_159629262.1,WP_192919075.1","WP_043267283.1,WP_009621388.1","WP_165888260.1,WP_165888262.1","WP_176808027.1,WP_176808029.1","WP_188156736.1,WP_188156734.1","WP_006249576.1,WP_006249574.1",WP_213434421.1,...,NaN,NaN,8485.756,8485.7560,NaN,NaN,NaN,NaN,NaN,NaN
977,50,73,0.112,WP_159630612.1,WP_061562514.1,WP_165889137.1,WP_176808743.1,WP_188156604.1,WP_006249213.1,WP_213433824.1,...,NaN,NaN,NaN,883.5764,NaN,NaN,NaN,NaN,NaN,NaN
1391,48,69,0.101,WP_159628569.1,WP_043315140.1,WP_165889526.1,WP_176807406.1,WP_188157006.1,WP_006249493.1,WP_213436741.1,...,NaN,NaN,NaN,6017.9190,23787.35,NaN,NaN,NaN,NaN,NaN
